# Codealong Notebook

Use this notebook as your "scratch pad" as you go through the course contents. Feel free to copy any example code and tweak it to get a better understanding of how it works!

Use the **+** button or `Insert` menu to add additional code cells as needed.

In [1]:
import openai
import os
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
except Exception:
    pass
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY is not set"

client = openai.OpenAI()

## Step 1

### Loading the 2022 Data with `pandas`

In [2]:
import requests

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
params = {
    "action": "query",
    "prop": "extracts",
    "exlimit": 1,
    "titles": "2022",
    "explaintext": 1,
    "formatversion": 2,
    "format": "json"
}
resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)
response_dict = resp.json()
response_dict["query"]["pages"][0]["extract"].split("\n")

['2022 (MMXXII) was a common year starting on Saturday of the Gregorian calendar, the 2022nd year of the Common Era (CE) and Anno Domini (AD) designations, the 22nd  year of the 3rd millennium and the 21st century, and the  3rd   year of the 2020s decade.  ',
 'The year began with another wave in the COVID-19 pandemic, with Omicron spreading rapidly and becoming the dominant variant of the SARS-CoV-2 virus worldwide. Tracking a decrease in cases and deaths, 2022 saw the removal of most COVID-19 restrictions and the reopening of international borders in the vast majority of countries, while the global rollout of COVID-19 vaccines continued. The global economic recovery from the pandemic continued, though many countries experienced an ongoing inflation surge; in response, many central banks raised their interest rates to landmark levels. The world population reached eight billion people in 2022. The year also witnessed numerous natural disasters, including two devastating Atlantic hurric

In [3]:
import pandas as pd

# Show more columns and set column width
pd.set_option('display.max_columns', None)        # Show all columns
pd.set_option('display.width', None)              # Auto-detect best width
pd.set_option('display.max_colwidth', None)       # Show full content of each cell

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = response_dict["query"]["pages"][0]["extract"].split("\n")

Here we try to clean up this text a bit, removing blank lines, headers and prefixing every line with a date


In [4]:
from dateutil.parser import parse

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")].reset_index(drop=True)

In [5]:
df

,text
0,"– 2022 (MMXXII) was a common year starting on Saturday of the Gregorian calendar, the 2022nd year of the Common Era (CE) and Anno Domini (AD) designations, the 22nd year of the 3rd millennium and the 21st century, and the 3rd year of the 2020s decade."
1,"– The year began with another wave in the COVID-19 pandemic, with Omicron spreading rapidly and becoming the dominant variant of the SARS-CoV-2 virus worldwide. Tracking a decrease in cases and deaths, 2022 saw the removal of most COVID-19 restrictions and the reopening of international borders in the vast majority of countries, while the global rollout of COVID-19 vaccines continued. The global economic recovery from the pandemic continued, though many countries experienced an ongoing inflation surge; in response, many central banks raised their interest rates to landmark levels. The world population reached eight billion people in 2022. The year also witnessed numerous natural disasters, including two devastating Atlantic hurricanes (Fiona and Ian), and the most powerful volcano eruption of the century so far. The later part of the year also saw the first public release of ChatGPT by OpenAI starting an arms race in artificial intelligence which increased in intensity into 2023, as well as the collapse of the cryptocurrency exchange FTX."
2,"– 2022 was also dominated by wars and armed conflicts. While escalations into the internal conflict in Myanmar and the Tigray War dominated the heightening of tensions within their regions and each caused over 10,000 deaths, 2022 was most notable for the Russian invasion of Ukraine, the largest armed conflict in Europe since World War II. The invasion caused the displacement of 15.7 million Ukrainians (8 million internally displaced persons and 7.7 million refugees), and led to international condemnations and sanctions and nuclear threats, the withdrawal of hundreds of companies from Russia, and the exclusion of Russia from major sporting events."
3,"– The ongoing Russian invasion of Ukraine escalated after Russian president Vladimir Putin announced a ""special military operation"", stating that it was to support the Russian-backed breakaway republics of Donetsk and Luhansk, whose paramilitary forces had been fighting Ukraine in the Donbas conflict since 2014."
4,"January 1 – France takes over the Presidency of the Council of the European Union, after the Slovenian presidency."
...,...
203,"December 24 – 2022 Fijian general election: The Parliament of Fiji elects Sitiveni Rabuka as Prime Minister of Fiji, defeating incumbent Prime Minister Frank Bainimarama after a contentious election."
204,"December 29 – Brazilian football legend Pelé dies at the age of 82, with his funeral being held at the Vila Belmiro stadium in Santos."
205,"December 31 – Former Pope Benedict XVI dies at the age of 95, with his funeral being held in St. Peter's Square, presided over by Pope Francis and Cardinal Giovanni Battista Re."
206,"December 7 – The world population was estimated to have reached 8 billion after 11 years on 15 November 2022. China was the most populous country throughout the year, followed by India. Africa was the fastest growing continent."


### Creating an Embeddings Index with `openai.Embedding`

Will use the [latest openai SDK](https://platform.openai.com/docs/guides/embeddings/embeddings?lang=python) `OpenAI().embeddings.create()` instead of the legacy `openai.Embedding.create()`

First we need to decide if/how we are going to chunk up each row of cleaned text in the dataframe. We might want to generate a single embedding for the entire text blob, or one per chunk.

🔍 Rule of Thumb:
If your use case benefits from semantic detail at a finer level (e.g., search, question-answering, or classification on parts of the text), then chunk it.
If you're summarizing or classifying the whole paragraph/document, then embedding the full text (if short enough) is okay.


🧠 Embedding Full Paragraph (One Embedding per Row):
✅ Good for:
Classification (e.g., spam/not spam, sentiment, topic) of the whole paragraph

When text is short enough to fit under token limits (e.g., <8000 tokens for OpenAI models like text-embedding-3-large)

❌ Problems:
Large paragraphs may get truncated or hit model token limits

Embedding gets "blurred" over multiple topics → less useful for similarity search


🧠 Chunking Paragraph into Smaller Segments (Multiple Embeddings per Row):
✅ Good for:
Semantic search / retrieval (RAG)

QA over documents (so you can retrieve relevant chunks, not whole documents)

Topic detection in specific sections

Fine-grained similarity comparison

🔧 Common Strategies:
Chunk by sentence or paragraph with overlapping context

Use fixed-size sliding windows (e.g., 256 or 512 tokens with 20% overlap)

Add metadata: source_id, chunk_id, etc.

**Let's try creating one embedding per sentence - the following code goes through each row of the original dataframe of paragraphs, breaking them up into individual sentences using a senetence tokenizer. The result is a new dataframe where each row is exactly one sentence. Finally an embedding is generated for each sentence and stored in the 'embedding' column. Notice that OpenAI currently generates embeddings of dimension 1536.**


In [5]:
# Example using sentence splitting and OpenAI embeddings
from nltk.tokenize import sent_tokenize
import openai
import nltk
import os

# Set NLTK data directory
nltk_data_dir = "./nltk_data"
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)


# Download the required NLTK resource
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab', download_dir=nltk_data_dir)

chunks = []
for idx, row in df.iterrows():
    sentences = sent_tokenize(row["text"])
    for i, sent in enumerate(sentences):
        chunks.append({"original_id": idx, "chunk_id": i, "text": sent})

sentences_df = pd.DataFrame(chunks) # chunk_df is a one sentence per row df


In [6]:
import openai
import pandas as pd
import time

# Assume you already have sentences_df with a 'text' column
# Set your model
EMBEDDING_MODEL = "text-embedding-3-small"

# Batching parameters
BATCH_SIZE = 100  # OpenAI allows up to 2048 inputs per request for `text-embedding-3-*`

# Helper: Call OpenAI and handle retries
def get_embeddings_batch(text_list, model=EMBEDDING_MODEL):
    try:
        response = openai.embeddings.create(input=text_list, model=model)
        return [e.embedding for e in response.data]
    except Exception as e:
        print("Retrying after error:", e)
        time.sleep(2)
        return get_embeddings_batch(text_list, model=model)

# Run batching
all_embeddings = []

for i in range(0, len(sentences_df), BATCH_SIZE):
    batch_texts = sentences_df["text"].iloc[i:i+BATCH_SIZE].tolist()
    print(f"Embedding batch {i//BATCH_SIZE + 1} of {len(sentences_df)//BATCH_SIZE + 1}...")
    batch_embeddings = get_embeddings_batch(batch_texts)
    all_embeddings.extend(batch_embeddings)

# Add to DataFrame
sentences_df["embeddings"] = all_embeddings



Embedding batch 1 of 3...
Embedding batch 2 of 3...
Embedding batch 3 of 3...


In [8]:
sentences_df.to_csv("embeddings.csv")

## Step 2

### Finding Relevant Data with Cosine Similarity

In [9]:
def get_embedding_for_userQ(question):

  # Generate the embedding response
  response = openai.embeddings.create(input=question, model=EMBEDDING_MODEL)

  # Extract the embeddings from the response
  return response.data[0].embedding


In [10]:
def cosine_similarity(a, b):
    import numpy as np
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [11]:
# calculate cosine distances between userQ and each embedding in df, then sort
def get_rows_sorted_by_relevance(q, df):
  Qe = get_embedding_for_userQ(q)
  distances = [1-cosine_similarity(e, Qe) for e in sentences_df["embeddings"]]
  df["distances"] = distances
  df.sort_values(by="distances", ascending=True, inplace=True)
  return df


## Step 3

### Tokenizing with `tiktoken`

In [15]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
toks=tokenizer.encode("some text...")
print(len(toks))


3


### Composing a Custom Text Prompt

In [12]:
def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context:

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

## Step 4

### Getting a Custom Q&A Response with `openai.Completion`

In [19]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=150):
  """
  Given a question, a dataframe containing rows of text, and a maximum
  number of desired tokens in the prompt and response, return the
  answer to the question according to an OpenAI Completion model

  If the model produces an error, return an empty string
  """

  try:
      prompt = create_prompt(question, df, max_prompt_tokens)

      response = client.completions.create(
          model=COMPLETION_MODEL_NAME,
          prompt=prompt,
          max_tokens=max_answer_tokens
      )
      return response.choices[0].text.strip()
  except Exception as e:
      print(e)
      return ""

In [21]:
print(answer_question("Who owns Twitter?", sentences_df));

Elon Musk.
